### Second try to get this automatic finite difference generator

In [1]:
from sympy import *

### First set up a test system

In [2]:
template = Matrix([[0, 0, 0],
                   [-1, 1, 1],
                   [0, 0, 0]])
diff = (2, 0)
order = 2
boundary_diff = (1, 0)

diff_degree = diff[0] + diff[1]
m, n = template.shape
k = diff_degree + order # total expansion degree
x, y = symbols('x y')

### Order gridpoints as a vector, make a list consisting of tuples $(i, j)$ where $i$ is the $x$ offset and $j$ is the $y$ offset

In [3]:
if not m % 2 or not n % 2:
    raise ValueError("Template dimensions must be odd")         
if m is not n:
    raise ValueError("Template matrix must be square")
    
# Find center of template, marked by -1
center = None
for i in range(m):
    for j in range(n):
        if template[i, j] == -1:
            center = (i, j)
if not center:
    raise ValueError("Did not set center of template")

# Make list of gridpoints from center in the template
# Note that it's in ij index convention
gridpoints = []
for i in range(m):
    for j in range(n):
        if template[i, j] == 1:
            gridpoints.append((j - center[1], center[0] - i))

### Generate monomials in graded lexicographical order, see which place in the Taylor series the desired partial is

In [4]:
f = exp(x)*exp(y)
q = expand( f.series(x, 0, k).removeO().series(y, 0, k).removeO() )
q = sum(term for term in q.args 
        if degree(term, x) + degree(term, y) < k
        and degree(term, x) + degree(term, y) > 0)
q = poly(q, x, y)
diff_place = q.monoms(order='grlex').index(diff)
if boundary_diff:
    excluded_place = q.monoms(order='grlex').index(boundary_diff)
else:
    excluded_place = None

### Generate matrix corresponding to Taylor series system of equations

In [6]:
M = zeros(len(gridpoints), q.length())
boundary_diff_vec = zeros(len(gridpoints), 1)
gridpoint_idx = 0
for (i, j) in gridpoints:
    f = exp(i*x)*exp(j*y)
    p = expand( f.series(x, 0, k).removeO().series(y, 0, k).removeO() )
    p = sum(term for term in p.args 
            if degree(term, x) + degree(term, y) < k 
            and degree(term, x) + degree(term, y) > 0)
    p = poly(p, x, y)

    taylor_term_idx = 0
    for deg in q.monoms(order='grlex'):
        if taylor_term_idx == excluded_place:
            M[gridpoint_idx, taylor_term_idx] = 0
            boundary_diff_vec[gridpoint_idx] = -p.coeff_monomial(deg)
        else:
            M[gridpoint_idx, taylor_term_idx] = p.coeff_monomial(deg)
        taylor_term_idx += 1
    gridpoint_idx += 1

In [8]:
M_inv = M.pinv()
diff_vec = zeros(q.length(), 1)
diff_vec[diff_place] = 1
if M_inv*M*diff_vec == diff_vec:
    stencil = template
    gridpoint_idx = 0
    centerpoint_val = 0
    for i in range(m):
        for j in range(n):
            if template[i, j] == 1:
                stencil[i, j] = M_inv[diff_place, gridpoint_idx]
                centerpoint_val += M_inv[diff_place, gridpoint_idx]
                gridpoint_idx += 1
    stencil[center[0], center[1]] = -centerpoint_val
else:
    print("Could not make a stencil")

In [9]:
M_inv

Matrix([
[-6,  3/2],
[ 0,    0],
[ 0,    0],
[ 0,    0],
[ 4, -1/2],
[ 0,    0],
[ 0,    0],
[ 0,    0],
[ 0,    0]])

In [10]:
stencil

Matrix([
[   0, 0,    0],
[-7/2, 4, -1/2],
[   0, 0,    0]])

In [13]:
(M_inv*boundary_diff_vec)[diff_place]

-3